In [12]:
import gensim.models
import gensim.corpora
import gensim as gs
import pyLDAvis as pvis
import pyLDAvis.gensim
import gensim.models.coherencemodel
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import f1_score
from gensim.models import FastText
from sklearn.metrics import classification_report

In [21]:
def save_models(dataset_name, num_topics):
    print("loading topic data for", dataset_name)
    # load inputs and labels
    dataset = pd.read_csv("../cleaned/" + dataset_name + "_stems.csv").astype(str).values.tolist() 
    # remove placeholders from the stems dataset
    print("removing placeholders")
    for index, sample in enumerate(dataset): 
            dataset[index] = list(filter((" ").__ne__, sample))
    # create dic, copora and lda-model
    print("making dic")
    dic = gs.corpora.Dictionary(dataset)
    #dic.save("../models/test/" + dataset_name + "_dictionary")
    print("making corpus")
    corpus = [dic.doc2bow(sample) for sample in dataset]
    print("making lda")
    #lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus, id2word=dic, num_topics=num_topics, random_state=100, chunksize=100, passes=10, per_word_topics=True)#update_every=1, 
    #lda_model.save("../models/test/" + dataset_name + "_ldamodel")
    print("making fasttext")
    inputs = [" ".join(sentence) for sentence in dataset]
    vector_model = FastText(size=32, window=3, min_count=1)
    vector_model.build_vocab(inputs)  
    vector_model.train(sentences=inputs, total_examples=len(inputs), total_words=vector_model.corpus_total_words, epochs=10)
    vector_model.save("../models/test/" + dataset_name + "_fasttext")
    
def load_test(dataset_name):
    print("loading dic")
    dic = gs.corpora.Dictionary.load("../models/dictionary/" + dataset_name + "_dictionary")
    print("loading topic model")
    lda_model = gensim.models.ldamulticore.LdaMulticore.load("../models/topic_models/" + dataset_name + "_ldamodel")
    print("loading fasttext")
    vector_model = FastText.load("../models/word_embeddings/" + dataset_name + "_fasttext")
    
    

In [22]:
num_topics_dict = {
    "norm_tweet": 79,
    "norm_emotion": 186,
}

dataset_name = "norm_tweet"
#save_models(dataset_name, num_topics_dict[dataset_name])

load_test(dataset_name)

loading dic
loading topic model
loading fasttext
